In [1]:
import feather
import pandas as pd
import numpy as np
import pymc3 as pm
import matplotlib.pyplot as plt
import seaborn as sns
import theano.tensor as tt

%matplotlib inline
sns.set_context('paper')
sns.set_style('darkgrid')

path = '/Users/chirag/Desktop/pymc_data.feather'
df = feather.read_dataframe(path)



df1 = df[['pheno_T1_1', 'pheno_T2_1']]

In [2]:

x=df1

In [3]:

nclus=2

Count_Row=df1.shape[0] 


with pm.Model() as model:
    
    p = pm.Dirichlet('p', a=np.array([1., 1.]), shape=nclus)
    
    category = pm.Categorical('category', p=p, shape=Count_Row)


with model:
    sigma_1 = pm.Lognormal('sigma_1', np.zeros(2), np.ones(2), shape=2)
    nu_1 = pm.Uniform('nu_1', 0, 5)
    C_triu_1 = pm.LKJCorr('C_triu_1', nu_1, 2)
    C_1 = pm.Deterministic('C_1', tt.fill_diagonal(C_triu_1[np.zeros((2, 2), dtype=np.int64)], 1.))
    sigma_diag_1 = pm.Deterministic('sigma_mat_1', tt.nlinalg.diag(sigma_1))
    cov_1 = pm.Deterministic('cov_1', tt.nlinalg.matrix_dot(sigma_diag_1, C_1, sigma_diag_1))

    
with model:
    sigma_2 = pm.Lognormal('sigma_2', np.zeros(2), np.ones(2), shape=2)
    nu_2 = pm.Uniform('nu_2', 0, 5)
    C_triu_2 = pm.LKJCorr('C_triu_2', nu_2, 2)
    C_2 = pm.Deterministic('C_2', tt.fill_diagonal(C_triu_2[np.zeros((2, 2), dtype=np.int64)], 1.))
    sigma_diag_2 = pm.Deterministic('sigma_mat_2', tt.nlinalg.diag(sigma_2))
    cov_2 = pm.Deterministic('cov_2', tt.nlinalg.matrix_dot(sigma_diag_2, C_2, sigma_diag_2))    
    cov_both = tt.stack([cov_1,cov_2])

with model:
    mu_both = pm.MvNormal('mu', mu=np.zeros(2), shape=(2,2))
    x_ = pm.MvNormal('x', mu_both[category], cov_both[category], observed=x)


AssertionError: 

In [ ]:
with model:
    trace = pm.sample(500)


In [ ]:
pm.traceplot(trace);